## **Importando pacotes**

In [21]:
import numpy                 as np
import pandas                as pd
import matplotlib.pyplot     as plt
import seaborn               as sns
import sys


from sklearn.metrics 	     import precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.compose 	     import ColumnTransformer
from sklearn.pipeline 	     import Pipeline
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing   import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.impute          import KNNImputer

from sklearn.linear_model    import LogisticRegression
from sklearn.svm             import SVC
from sklearn.tree            import DecisionTreeClassifier
from sklearn.ensemble        import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier
from xgboost                 import XGBClassifier
from lightgbm                import LGBMClassifier
from catboost                import CatBoostClassifier
from sklearn.dummy           import DummyClassifier
from sklearn.neighbors       import KNeighborsClassifier

from sklearn.feature_selection import SelectKBest
from category_encoders.target_encoder import TargetEncoder


sys.path.append('../utils')

from modelcrafterclass import ModelCrafter
#import warnings
#warnings.filterwarnings("ignore")

## **Carregando os dados**

In [22]:
data = pd.read_csv('../datasets_for_ml/dataset_for_train.csv')

In [23]:
data.head()

,Client,data_recente,target,qt_faltas,moda_staff_faltante,moda_servico_faltante,moda_servico_cancelado,moda_staff_cancelado,antecedencia,qt_cancelamentos,moda_staff_prestou_servico,moda_dia,qte_servicos_por_dia,media,mediana,desvio_padrao,min,max,qte_servico_recebido
0,KERT01,2018-06-20,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,JJ,Tuesday,1.5,84.666667,82.0,16.165808,70.0,102.0,3
1,COOM01,2018-06-15,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,SINEAD,Thursday,1.0,70.000000,70.0,NaN,70.0,70.0,1
2,PEDM01,2018-06-09,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,BECKY,Saturday,1.0,60.000000,60.0,NaN,60.0,60.0,1
3,BAIS01,2018-06-09,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,nenhum,nenhum,NaN,NaN,NaN,NaN,NaN,NaN,0
4,FRAL01,2018-06-09,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,nenhum,nenhum,NaN,NaN,NaN,NaN,NaN,NaN,0


Apesar de haver valores faltantes, esses não são advindos de erros ou problemas. Na realidade, esses valores nulos tem seus significados dentro do contexto do nosso problema.

Por exemplo, um registro com a media faltante está associado a um novo cliente. Antecedência faltante indica que um cliente nunca fez um cancelamento e assim por diante.

Portanto, devemos achar uma maneira de introduzir essas informações na nossa base de dados

In [24]:
data.isna().sum().to_frame('qte. missing').sort_values(by='qte. missing',ascending=False)

,qte. missing
antecedencia,734
desvio_padrao,590
max,461
min,461
mediana,461
media,461
qte_servicos_por_dia,461
Client,0
moda_staff_prestou_servico,0
moda_dia,0


Como todos os valores nulos são numéricos, vamos substitui-los por -1

In [25]:
data = data.fillna(-1)

Vamos adicionar uma nova coluna, indicando se temos um novo cliente. Para fazer isso vamos tomar como base a variável média (poderia ser qualquer outra). Todas as instâncias com media faltante receberão 1 e as demais 0

In [26]:
data['novo_cliente'] = data.apply(lambda x: 1 if x['media'] == -1 else 0,axis=1)

# **Breve EDA**

Ao todo há 798 registros, ou seja, 798 clientes únicos

In [27]:
data.shape

(798, 20)

Estamos lidando com um problema de classes desbalanceadas onde 751 (94%) clientes seguiram as políticas do salão enquanto 47 (5.9%) não seguiram

In [28]:
pd.concat([data['target'].value_counts().to_frame('Absoluto'),round(data['target'].value_counts(normalize=True).mul(100).to_frame('%'),2)],axis=1)

,Absoluto,%
target,,
0,751,94.11
1,47,5.89


Observamos que 461 (58%) são novos clientes e 337 (42%) são clientes com algum histórico.

In [29]:
pd.concat([data['novo_cliente'].value_counts().to_frame('Absoluto'),round(data['novo_cliente'].value_counts(normalize=True).mul(100).to_frame('%'),2)],axis=1)

,Absoluto,%
novo_cliente,,
1,461,57.77
0,337,42.23


Da observação acima, podemos notar um problema. Mais da metade do dataset são novos cliente e, portanto, não tem nenhum histórico associado.

Clientes sem histórico podem ser um problema, pois a única informação que adquirimos sobre eles está no momento da reserva, o que pode não ser muito informativo.

> Para lidar com isso nós vamos treinar um modelo com todos os clientes (novos e antigos) e retreinar esse modelo para apenas clientes antigos. Avaliaremos os resultados para ambos os modelos e concluiremos em qual linha vamos seguir

## **Definições**

Devemos estabelecer as métricas que vamos utilizar para a avaliação dos modelos. Vamos utilizar as três métricas citadas a seguir:

1. **Precision Score;**
2. **Recall Score;**
3. **F1 Score;**

A questão central do problema consiste em identificar faltas ou cancelamentos que fuja das políticas do salão. Tais ações acarretam em custos para o salão, desse modo, devemos selecionar se um cliente vai ou não seguir as políticas.

Vamos focar em maximizar a métrica f1-score, entretanto, no caso limite da f1 vamos focar em maximizar o recall.

Maximizando a f1 nós estamos maximizando precision e recall e maximizando o recall nós estamos interessados em identificar o máximo de clientes que não vão seguir com as políticas, entretanto, gostaríamos de fazer isso sem perder muito da precisão. 

# **Modelagem**

Definindo as features e o target

In [30]:
X = data.drop(['Client','data_recente','target'],axis=1)

y = data['target']

Definindo as variáveis numéricas e categóricas

In [31]:
categoricas = X.select_dtypes(include ='object').columns

numericas = X.select_dtypes(exclude ='object').columns

Instanciando o modelcrafter

In [32]:
modelcrafter = ModelCrafter(folds = 5)

Adicionando os modelos de classificação na estrutura

In [33]:
modelos = [('regressao_logistica', LogisticRegression(class_weight='balanced', max_iter=100000)),
           ('svc',SVC(class_weight='balanced')),
           ('knn',KNeighborsClassifier()),
           ('arvore',DecisionTreeClassifier(class_weight='balanced', random_state = 0 )),
           ('floresta',RandomForestClassifier(n_estimators=500,min_samples_split=10,class_weight='balanced',random_state = 0)),
           ('adaboost',AdaBoostClassifier(random_state = 0)),
           ('gradientboost',GradientBoostingClassifier(random_state = 0)),
           ('xgboost',XGBClassifier(random_state = 0)),
           ('lgbm',LGBMClassifier(class_weight='balanced',force_row_wise=True)),
           ('catboost',CatBoostClassifier(verbose=0)),
           ('dummy',DummyClassifier(strategy='uniform'))] 

modelcrafter.AddModel(modelos = modelos)

Estabelecendo uma pipeline inicial para os modelos

In [34]:
numerical_transform = Pipeline([("scaler",StandardScaler())])

categorical_transform = Pipeline([('encoder',TargetEncoder()), 
                                  ('scaler',StandardScaler())])

preprocessor = ColumnTransformer([('categorical',categorical_transform, categoricas), 
                                  ('numeric', numerical_transform,numericas)],
                                remainder='passthrough')


model_pipeline = Pipeline([('preprocessor',preprocessor)])

model_pipeline.set_output(transform='pandas')

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical',
                                                  Pipeline(steps=[('encoder',
                                                                   TargetEncoder()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['moda_staff_faltante', 'moda_servico_faltante',
       'moda_servico_cancelado', 'moda_staff_cancelado',
       'moda_staff_prestou_servico', 'moda_dia'],
      dtype='object')),
                                                 ('numeric',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['qt_faltas', 'antecedencia', 'qt_cancelamentos', 'qte_servicos_por_dia',
       'media', 'mediana', 'desvio_padrao', 'min', 'max',
       'qte_servico_recebido', 'novo_cliente'],
      dtype='object'))]))])

Separando em treino e teste

In [35]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

Realizando uma baseline

In [36]:
baseline = modelcrafter.ValidacaoCruzada(X_train,y_train,model_pipeline)

baseline.sort_values(by='f1',ascending=False)

-----regressao_logistica-----
-----svc-----
-----knn-----


/home/alexandre/Área de Trabalho/projetos_ds/desonra/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alexandre/Área de Trabalho/projetos_ds/desonra/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alexandre/Área de Trabalho/projetos_ds/desonra/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

-----arvore-----
-----floresta-----
-----adaboost-----
-----gradientboost-----
-----xgboost-----


/home/alexandre/Área de Trabalho/projetos_ds/desonra/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


-----lgbm-----
[LightGBM] [Info] Number of positive: 23, number of negative: 423
[LightGBM] [Info] Total Bins 225
[LightGBM] [Info] Number of data points in the train set: 446, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

/home/alexandre/Área de Trabalho/projetos_ds/desonra/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alexandre/Área de Trabalho/projetos_ds/desonra/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alexandre/Área de Trabalho/projetos_ds/desonra/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

-----dummy-----


,precision,recall,f1
floresta,0.150000,0.150000,0.147143
svc,0.107619,0.236667,0.139942
lgbm,0.126007,0.140000,0.119227
dummy,0.066651,0.636667,0.118920
regressao_logistica,0.081099,0.270000,0.116222
adaboost,0.066667,0.033333,0.044444
arvore,0.033333,0.020000,0.025000
knn,0.000000,0.000000,0.000000
gradientboost,0.000000,0.000000,0.000000
xgboost,0.000000,0.000000,0.000000


Notamos que alguns modelos tem todas as métricas iguais a zero.

A seguir as expressões para precision e recall:

$Precision = \dfrac{VP}{VP+FP}$

$Recall = \dfrac{VP}{VP+FN}$

A única maneira de obtermos precision e recall iguais a zero é obtendo os verdadeiros positivos iguais a zero. Além disso, nos warnings observamos que há uma possível divisão por zero no precision. Isso ocorre somente se os verdadeiros positivos e os falsos positivos forem iguais a zero, portanto, podemos entender que esses modelos estão estimando somente 0's.

Poderíamos alterar os hiperparâmetros desses modelos afim de conseguir obter resultados mais consistentes. Entretanto, por agora, podemos seguir com os modelos que não deram nenhum problema. 

In [38]:
#modelcrafter = ModelCrafter()
#estimadores = [('regressao_logistica', LogisticRegression(class_weight='balanced', max_iter=100000)),
#           ('svc',SVC(kernel='rbf',C=0.5,class_weight='balanced')),
#           ('floresta',RandomForestClassifier(n_estimators=500,min_samples_split=10,class_weight='balanced',random_state = 0))]
#
#
#modelos = [('regressao_logistica', LogisticRegression(class_weight='balanced', max_iter=100000)),
#           ('svc',SVC(kernel='rbf',C=0.5,class_weight='balanced')),
#           ('floresta',RandomForestClassifier(n_estimators=500,min_samples_split=10,class_weight='balanced',random_state = 0)),
#           ('stack',StackingClassifier(estimators=estimadores,passthrough=True))] 
#
#modelcrafter.AddModel(modelos = modelos)
#modelcrafter.ValidacaoCruzada(X_train,y_train,model_pipeline)